In [ ]:
#This notebook aims to identify and rank trending NFT collections on the Base, considering both minting activity and creator influence. 
#We'll use on-chain data to build a graph of interactions between minters and collections, then apply the eigentrust algorithm
#to determine the most influential and trending collections.

In [1]:
# Data Loading and Preprocessing
# Note: This dataset is limited to 100,000 records from the last 30 days
import pandas as pd
import numpy as np
from dune_client.client import DuneClient
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize Dune client
dune = DuneClient(os.getenv("DUNE_API_KEY"))

# Fetch the query results
query_result = dune.get_latest_result(3997754)

# Convert to DataFrame
df = pd.DataFrame(query_result.result.rows)

print(df.head())
print(df.info())

# Data Limitations and Considerations:
# 1. Limited to 100,000 records, which may not represent the entire Base NFT ecosystem
# 2. Data is from the last 30 days, so historical trends beyond this period are not captured
# 3. Only considers minting activity, not secondary market transactions

                              creator_address              first_mint_time  \
0  0x827922686190790b37229fd06084350e74485b72  2024-07-19 00:02:17.000 UTC   
1  0x827922686190790b37229fd06084350e74485b72  2024-07-19 00:16:43.000 UTC   
2  0x827922686190790b37229fd06084350e74485b72  2024-07-26 19:04:29.000 UTC   
3  0x827922686190790b37229fd06084350e74485b72  2024-07-19 01:32:39.000 UTC   
4  0x827922686190790b37229fd06084350e74485b72  2024-07-19 00:16:33.000 UTC   

                last_mint_time  mint_count  \
0  2024-08-18 09:50:05.000 UTC        9346   
1  2024-08-18 10:17:21.000 UTC        2888   
2  2024-08-18 09:50:03.000 UTC        4414   
3  2024-08-18 02:03:17.000 UTC         418   
4  2024-08-17 16:08:53.000 UTC         633   

                                       minter  \
0  0xc0d28f068f5483e18c4da8025a9a97dddc8d42f2   
1  0x1f2473e8ff13905544f761651b91dcced067b95f   
2  0xc5f32bb698412eb964a2e29193107832c38c70f6   
3  0x89b585df208c727829232d892a50806c3a20a4e6   
4  0xa15c

In [2]:
# Data Processing
import pandas as pd
import numpy as np
from collections import defaultdict
import math

# Convert time columns to datetime
df['first_mint_time'] = pd.to_datetime(df['first_mint_time'])
df['last_mint_time'] = pd.to_datetime(df['last_mint_time'])

# Ensure numeric columns are of the right type
df['mint_count'] = df['mint_count'].astype(int)
df['total_amount_usd'] = df['total_amount_usd'].astype(float)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype              
---  ------                --------------   -----              
 0   creator_address       100000 non-null  object             
 1   first_mint_time       100000 non-null  datetime64[ns, UTC]
 2   last_mint_time        100000 non-null  datetime64[ns, UTC]
 3   mint_count            100000 non-null  int64              
 4   minter                100000 non-null  object             
 5   nft_contract_address  100000 non-null  object             
 6   total_amount_usd      100000 non-null  float64            
dtypes: datetime64[ns, UTC](2), float64(1), int64(1), object(3)
memory usage: 5.3+ MB
None


In [3]:
# Calculate localtrust scores
def calculate_localtrust(df):
    # Create dictionaries to store minters and total value for each collection
    collection_minters = defaultdict(set)
    collection_total_value = defaultdict(float)
    
    for _, row in df.iterrows():
        collection_minters[row['nft_contract_address']].add(row['minter'])
        collection_total_value[row['nft_contract_address']] += row['total_amount_usd']
    
    collections = list(collection_minters.keys())
    n = len(collections)
    localtrust = np.zeros((n, n))

     # Calculate localtrust scores
    for i in range(n):
        for j in range(i+1, n):
            shared_minters = len(collection_minters[collections[i]] & collection_minters[collections[j]])
            if shared_minters > 0:
                shared_value = min(collection_total_value[collections[i]], collection_total_value[collections[j]])
                score = shared_minters * math.log(1 + shared_value)
                localtrust[i][j] = score
                localtrust[j][i] = score  # Matrix is symmetric
    
    return localtrust, collections

localtrust, collections = calculate_localtrust(df)
print("Localtrust matrix shape:", localtrust.shape)

Localtrust matrix shape: (3208, 3208)


In [26]:
# Calculate pretrust scores based on creator activity and influence
def calculate_pretrust(df):
    creator_scores = defaultdict(lambda: {'total_value': 0, 'unique_minters': set(), 'collections': set(), 'first_mint': None, 'last_mint': None})

    # Aggregate data for each creator
    for _, row in df.iterrows():
        creator = row['creator_address']
        creator_scores[creator]['total_value'] += row['total_amount_usd']
        creator_scores[creator]['unique_minters'].add(row['minter'])
        creator_scores[creator]['collections'].add(row['nft_contract_address'])

        # Track the first and last mint times for each creator
        if creator_scores[creator]['first_mint'] is None or row['first_mint_time'] < creator_scores[creator]['first_mint']:
            creator_scores[creator]['first_mint'] = row['first_mint_time']
        
        if creator_scores[creator]['last_mint'] is None or row['last_mint_time'] > creator_scores[creator]['last_mint']:
            creator_scores[creator]['last_mint'] = row['last_mint_time']
            
    # Calculate the final score for each creator
    for creator in creator_scores:
        duration = (creator_scores[creator]['last_mint'] - creator_scores[creator]['first_mint']).total_seconds() / (24 * 3600) + 1
        creator_scores[creator]['score'] = (
            math.log(1 + creator_scores[creator]['total_value']) *
            len(creator_scores[creator]['unique_minters']) *
            len(creator_scores[creator]['collections']) *
            math.log(1 + duration)
        )
    
    return creator_scores

creator_scores = calculate_pretrust(df)

# Print stats about creator scores
print(f"Number of unique creators: {len(creator_scores)}")
print(f"Max creator score: {max(score['score'] for score in creator_scores.values())}")
print(f"Min creator score: {min(score['score'] for score in creator_scores.values())}")

# Map creator scores to collections
collection_to_idx = {collection: idx for idx, collection in enumerate(collections)}
pretrust = np.zeros(len(collections))

for _, row in df.iterrows():
    collection_idx = collection_to_idx[row['nft_contract_address']]
    creator = row['creator_address']
    pretrust[collection_idx] = creator_scores[creator]['score']

print("Pretrust vector shape:", pretrust.shape)
print(f"Number of non-zero pretrust scores: {np.count_nonzero(pretrust)}")
print(f"Max pretrust score: {np.max(pretrust)}")
print(f"Min non-zero pretrust score: {np.min(pretrust[pretrust > 0])}")

Number of unique creators: 2390
Max creator score: 11794421.713476304
Min creator score: 1.142975580056842
Pretrust vector shape: (3208,)
Number of non-zero pretrust scores: 3208
Max pretrust score: 11794421.713476304
Min non-zero pretrust score: 1.142975580056842


In [5]:
import numpy as np
import pandas as pd
from collections import defaultdict
import math
from openrank_sdk.eigentrust_client import EigenTrust, IJV, IV


def calculate_localtrust(df):
    collection_minters = defaultdict(set)
    collection_total_value = defaultdict(float)
    
    for _, row in df.iterrows():
        collection_minters[row['nft_contract_address']].add(row['minter'])
        collection_total_value[row['nft_contract_address']] += row['total_amount_usd']
    
    collections = list(collection_minters.keys())
    n = len(collections)
    localtrust = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i+1, n):
            shared_minters = len(collection_minters[collections[i]] & collection_minters[collections[j]])
            if shared_minters > 0:
                shared_value = min(collection_total_value[collections[i]], collection_total_value[collections[j]])
                score = shared_minters * math.log(1 + shared_value)
                localtrust[i][j] = score
                localtrust[j][i] = score
    
    return localtrust, collections

def calculate_pretrust(df, collections):
    pretrust = np.zeros(len(collections))
    for _, row in df.iterrows():
        if row['nft_contract_address'] in collections:
            index = collections.index(row['nft_contract_address'])
            pretrust[index] += row['total_amount_usd']
    return pretrust

# Calculate localtrust and collections
localtrust, collections = calculate_localtrust(df)

# Calculate pretrust
pretrust = calculate_pretrust(df, collections)

# Get the indices of non-zero entries in localtrust
localtrust_indices = set(localtrust.nonzero()[0]) | set(localtrust.nonzero()[1])

# Filter pretrust to only include indices present in localtrust
filtered_pretrust = [IV(i=i, v=v) for i, v in enumerate(pretrust) if i in localtrust_indices and v > 0]

# Create localtrust_ijv using only the filtered indices
localtrust_ijv = [IJV(i=i, j=j, v=v) for i, j in zip(*localtrust.nonzero()) for v in [localtrust[i, j]] if i in localtrust_indices and j in localtrust_indices]

print(f"Number of localtrust entries: {len(localtrust_ijv)}")
print(f"Number of pretrust entries: {len(filtered_pretrust)}")

# Initialize EigenTrust client
et_client = EigenTrust(host_url='https://ek-go-eigentrust.k3l.io')

# Run eigentrust algorithm
result = et_client.run_eigentrust(localtrust_ijv, filtered_pretrust, alpha=0.5)

# Print the structure of the result
print("Result structure:")
print(type(result))
print(f"Length of result: {len(result)}")
if len(result) > 0:
    print(f"Type of first element: {type(result[0])}")
    print(f"Keys in first element: {result[0].keys()}")

# Convert result to a dictionary
collection_scores = {}
for item in result:
    if 'i' in item and 'v' in item:
        index = int(item['i'])  # Convert np.int64 to regular int
        if index < len(collections):
            collection_scores[collections[index]] = float(item['v'])  # Convert to regular float

# Create a DataFrame with the results
result_df = pd.DataFrame(list(collection_scores.items()), columns=['nft_contract_address', 'score'])
result_df = result_df.sort_values('score', ascending=False).reset_index(drop=True)
print(result_df.head(10))

# Print some stats about the scores
if not result_df.empty:
    print("\nScore statistics:")
    print(result_df['score'].describe())
else:
    print("\nNo valid scores were generated.")

Number of localtrust entries: 64850
Number of pretrust entries: 2014


/Users/yulia/Library/Python/3.9/lib/python/site-packages/openrank_sdk/eigentrust_client.py:317: UserWarning: Defaulting to the 'raw' score scale. The default scale will change to 'log' in a future version; add score='raw' to keep the current behavior (and silence this warning)
  warnings.warn(
2024-08-19 11:48:14,120 INFO httpx HTTP Request: POST https://ek-go-eigentrust.k3l.io/basic/v1/compute "HTTP/1.1 200 OK"
2024-08-19 11:48:14,350 INFO root eigentrust compute took 5.531003416000004 secs


Result structure:
<class 'list'>
Length of result: 2014
Type of first element: <class 'dict'>
Keys in first element: dict_keys(['i', 'v'])
                         nft_contract_address     score
0  0x827922686190790b37229fd06084350e74485b72  0.511420
1  0x03a520b32c04bf3beef7beb72e919cf822ed34f1  0.133175
2  0xebf418fe2512e7e6bd9b87a8f0f294acdc67e6b4  0.046130
3  0x76fea18dca768c27afc3a32122c6b808c0ad9b06  0.039236
4  0x46a15b0b27311cedf172ab29e4f4766fbe7f4364  0.033661
5  0x8453000bab46eaaa842b1a6b695d8fa9ef0b144f  0.028829
6  0x01df6fb6a28a89d6bfa53b2b3f20644abf417678  0.020748
7  0xc2462f03920d47fc5b9e2c5f0ba5d2ded058fd78  0.016772
8  0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7  0.016525
9  0x80c7dd17b01855a6d2347444a0fcc36136a314de  0.015097

Score statistics:
count    2.014000e+03
mean     4.965243e-04
std      1.192123e-02
min      5.159935e-10
25%      9.571754e-08
50%      1.004971e-06
75%      5.937998e-06
max      5.114195e-01
Name: score, dtype: float64


In [6]:
# Output feed with trending NFT Collections on Base
# IMPORTANT LIMITATIONS:
# 1. Dataset Limitation: This analysis is based on a dataset limited to 100,000 records. Results do not reflect the entire NFT ecosystem on Base.
# 2. Time Frame: The data covers only the last 30 days, missing longer-term trends.
# 3. Sampling Bias: The 100,000 record limit may introduce bias, possibly over-representing high-value or recent transactions.
# 4. Incomplete Picture: Due to data limitations, some influential collections/creators might be underrepresented or missing entirely from this analysis.
def get_collection_details(collection_address):
    collection_data = df[df['nft_contract_address'] == collection_address]
    total_mints = collection_data['mint_count'].sum()
    total_value = collection_data['total_amount_usd'].sum()
    unique_minters = collection_data['minter'].nunique()
    creator = collection_data['creator_address'].iloc[0]
    return {
        'address': collection_address,
        'total_mints': total_mints,
        'total_value_usd': total_value,
        'unique_minters': unique_minters,
        'creator': creator
    }

print("Top 10 Trending NFT Collections on Base:")
for i, row in result_df.head(10).iterrows():
    details = get_collection_details(row['nft_contract_address'])
    print(f"{i+1}. Collection: {details['address']}")
    print(f"   Score: {row['score']:.6f}")
    print(f"   Creator: {details['creator']}")
    print(f"   Total Mints: {details['total_mints']}")
    print(f"   Total Value (USD): ${details['total_value_usd']:,.2f}")
    print(f"   Unique Minters: {details['unique_minters']}")
    print()
print("DISCLAIMER: This analysis is based on a limited dataset of 100,000 records over the last 30 days.")
print("The results do not accurately represent the full scope of NFT activity on the Base.")

Top 10 Trending NFT Collections on Base:
1. Collection: 0x827922686190790b37229fd06084350e74485b72
   Score: 0.511420
   Creator: 0x827922686190790b37229fd06084350e74485b72
   Total Mints: 49831
   Total Value (USD): $8,645,729,184.83
   Unique Minters: 3346

2. Collection: 0x03a520b32c04bf3beef7beb72e919cf822ed34f1
   Score: 0.133175
   Creator: 0x03a520b32c04bf3beef7beb72e919cf822ed34f1
   Total Mints: 50220
   Total Value (USD): $146,989,577.99
   Unique Minters: 9048

3. Collection: 0xebf418fe2512e7e6bd9b87a8f0f294acdc67e6b4
   Score: 0.046130
   Creator: 0xebf418fe2512e7e6bd9b87a8f0f294acdc67e6b4
   Total Mints: 1675
   Total Value (USD): $4,390,715.48
   Unique Minters: 1216

4. Collection: 0x76fea18dca768c27afc3a32122c6b808c0ad9b06
   Score: 0.039236
   Creator: 0x1aed60a97192157fda7fb26267a439d523d09c5e
   Total Mints: 15903
   Total Value (USD): $114,554.34
   Unique Minters: 15237

5. Collection: 0x46a15b0b27311cedf172ab29e4f4766fbe7f4364
   Score: 0.033661
   Creator: 0x46a1